<a href="https://colab.research.google.com/github/grissharrisdennis/Machine-learning-Projects/blob/main/Similarity_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Similarity Prediction

Molecular similarity assessments using machine learning.
Useful for the prediction of molecular similarity evaluations by humans.

Molecular similarity is an impressively broad topic with many implications in several areas of chemistry. Its roots lie in the paradigm that ‘similar molecules have similar properties’. For this reason, methods for determining molecular similarity find wide application in pharmaceutical companies, e.g., in the context of structure-activity relationships. The similarity evaluation is also used in the field of chemical legislation, specifically in the procedure to judge if a new molecule can obtain the status of orphan drug with the consequent financial benefits. For this procedure, the European Medicines Agency uses experts’ judgments. It is clear that the perception of the similarity depends on the observer, so the development of models to reproduce the human perception is useful.

The dataset was created by Enrico Gandini during his PhD at Università degli Studi di Milano.

[Link to the dataset](https://archive.ics.uci.edu/dataset/750/similarity+prediction-1)[click here]

# Acknowledgements
Gandini, Enrico, Gilles Marcou, Fanny Bonachera, Alexandre Varnek, Stefano Pieraccini, and Maurizio Sironi. 2022.
 "Molecular Similarity Perception Based on Machine-Learning Models" International Journal of Molecular Sciences 23, no. 11: 6114. https://doi.org/10.3390/ijms23116114



In [ ]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 33.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#train_data_images_path='/content/drive/My Drive/dataset_Similarity_Prediction/original_training_set/images_2D'
#train_data_images=os.listdir('/content/drive/My Drive/dataset_Similarity_Prediction/original_training_set/images_2D')
#train_data_conformers=os.listdir('/content/drive/My Drive/dataset_Similarity_Prediction/original_training_set/conformers_3D')
train_data=pd.read_csv('/content/drive/My Drive/dataset_Similarity_Prediction/original_training_set/original_training_set.csv')

In [ ]:
test_data=pd.read_csv('/content/drive/My Drive/dataset_Similarity_Prediction/new_dataset/new_dataset.csv')

In [ ]:
ids = train_data['id_pair']
smiles_a = train_data['curated_smiles_molecule_a']
smiles_b = train_data['curated_smiles_molecule_b']
tanimoto_coefficients = train_data['tanimoto_cdk_Extended']
tanimoto_combo = train_data['TanimotoCombo']
frac_similar = train_data['frac_similar']


In [ ]:
tids = test_data['id_pair']
tsmiles_a = test_data['curated_smiles_molecule_a']
tsmiles_b = test_data['curated_smiles_molecule_b']
ttanimoto_coefficients = test_data['tanimoto_cdk_Extended']
ttanimoto_combo = test_data['TanimotoCombo']
tfrac_similar = test_data['frac_similar']

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

def convert_smiles_to_fingerprints(smiles):
    # Convert SMILES to RDKit Mol object
    molecule = Chem.MolFromSmiles(smiles)

    # Generate molecular fingerprint (Morgan fingerprint in this example)
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(molecule, 2)

    # Convert RDKit fingerprint object to bit vector
    fingerprint_bitvector = list(fingerprint.ToBitString())

    return fingerprint_bitvector


In [ ]:
smiles_list = []
for i, j in zip(smiles_a, smiles_b):
    fingerprint_i = convert_smiles_to_fingerprints(i)
    fingerprint_j = convert_smiles_to_fingerprints(j)

    smiles_list.append([fingerprint_i, fingerprint_j])

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Flatten the list of fingerprints
flattened_smiles_list = np.array(smiles_list).reshape(len(smiles_list), -1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(flattened_smiles_list, frac_similar, test_size=0.2, random_state=42)

# Example preprocessing for numerical and categorical data
numerical_scaler = StandardScaler()
X_train_numerical = numerical_scaler.fit_transform(X_train)
X_val_numerical = numerical_scaler.transform(X_val)

# Combine y_train and y_val for label encoding
combined_labels = np.concatenate([y_train, y_val])

# Encode the combined labels
label_encoder = LabelEncoder()
combined_labels_encoded = label_encoder.fit_transform(combined_labels)

# Use the label encoder to transform both training and validation labels
y_train_encoded = combined_labels_encoded[:len(y_train)]
y_val_encoded = combined_labels_encoded[len(y_train):]

# Determine the input shape based on the length of flattened fingerprint vectors
input_shape = len(flattened_smiles_list[0])

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(input_shape,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train_numerical, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_val_numerical, y_val_encoded))


Epoch 1/10
3/3 [==============================] - 1s 125ms/step - loss: 1349.2025 - mae: 28.9062 - val_loss: 1393.4646 - val_mae: 30.4102
Epoch 2/10
3/3 [==============================] - 0s 34ms/step - loss: 1172.9910 - mae: 26.5454 - val_loss: 1308.0671 - val_mae: 29.3011
Epoch 3/10
3/3 [==============================] - 0s 33ms/step - loss: 1033.8851 - mae: 24.6075 - val_loss: 1212.0670 - val_mae: 28.0593
Epoch 4/10
3/3 [==============================] - 0s 26ms/step - loss: 877.9229 - mae: 22.3563 - val_loss: 1103.5017 - val_mae: 26.8137
Epoch 5/10
3/3 [==============================] - 0s 37ms/step - loss: 719.6597 - mae: 19.8100 - val_loss: 981.8629 - val_mae: 25.4241
Epoch 6/10
3/3 [==============================] - 0s 23ms/step - loss: 541.2590 - mae: 16.7363 - val_loss: 854.6725 - val_mae: 23.8218
Epoch 7/10
3/3 [==============================] - 0s 24ms/step - loss: 383.4366 - mae: 13.4429 - val_loss: 722.4869 - val_mae: 22.2229
Epoch 8/10
3/3 [==============================]

In [24]:
tsmiles_list = []
for i, j in zip(tsmiles_a, tsmiles_b):
    fingerprint_i = convert_smiles_to_fingerprints(i)
    fingerprint_j = convert_smiles_to_fingerprints(j)
    tsmiles_list.append([fingerprint_i, fingerprint_j])

In [28]:
# Flatten the list of fingerprints for test data
tflattened_smiles_list = np.array(tsmiles_list).reshape(len(tsmiles_list), -1)

# Preprocess numerical features
X_test_numerical = numerical_scaler.transform(tflattened_smiles_list)

In [29]:
predictions = model.predict(X_test_numerical)

4/4 [==============================] - 0s 4ms/step


In [ ]:
import cairosvg
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Concatenate

In [ ]:
!pip install cairosvg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 886.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
def extract_images_from_svg(svg_file):
    png_file = svg_file.replace('.svg', '.png')

    # Convert SVG to PNG using CairoSVG
    cairosvg.svg2png(url=svg_file, write_to=png_file)
    return png_file

In [ ]:
svg_files = [os.path.join(train_data_images_path, file) for file in train_data_images if file.endswith('.svg')]
a_images=[]
b_images=[]
for svg_file in svg_files:
    png_img_path = extract_images_from_svg(svg_file)
    #png_img_path=preprocess_image(png_img_p)
    if png_img_path[104:105]=='a':
      a_images.append(png_img_path)
    elif png_img_path[104:105]=='b':
      b_images.append(png_img_path)

In [ ]:
images_data = [[] for _ in range(101)]  # Initialize a list of lists for each number (0-100)
for img_path in a_images + b_images:
    molecule_num = int(img_path.split('_')[-1][:3])  # Extract the molecule number
    images_data[molecule_num].append(img_path)
images_data.pop(0)

In [ ]:
# Convert images_data to numpy arrays for input to the model
image_pairs = np.array(images_data)  # Assuming images_data contains pairs of preprocessed images
frac_similar_values = np.array(train_data['frac_similar'])  # Assuming frac_similar is a column from train_data
print(image_pairs)
# Assuming your image_pairs are of shape (num_samples, 2, img_width, img_height, img_channels)
# Reshape to (num_samples, img_width, img_height, img_channels) for each image in the pair
#image_pairs = image_pairs.reshape(-1, 2, img_width, img_height, img_channels)
